# Stock Price Modelling Regression Experiment 1.0

Build baseline models with default parameters. Evaluate their performance. Save all models

# Modelling Regression Experiment #1.0

- Split data into Train / Validation / Test set
- Create Baseline Models
    - 1) Arima
        - Test for stationarity
        - Find p, q, d terms
        - Model Building
    - 2) Multivariate Arima
        - Causality investigation.
        - Test for stationary
        - Model Building.
        - Test for residuals (errors).
        - Forecasting
        - Model evaluation
    - 3) Random Forest
        - Model Building.
        - Model evaluation against the test set
    - 4) XGBoost
        - Model Building.
        - Model evaluation against the test set
    - 5) kNN
        - Model Building.
        - Model evaluation against the test set
    - 6) SVM
        - Model Building.
        - Model evaluation against the test set
    - 7) LSTM
        - Model Building.
        - Model evaluation against the test set
- Compare their performace

# 1 Install more libraries

In [ ]:
!pip install -q yfinance==0.2.28
!pip install -q pmdarima

# 2 Loading the libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

# ML libraries
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# 3 Download stock data

In [ ]:
def get_stock_data(stock_name):
    
    # For time stamps
    from datetime import datetime
    import yfinance as yf
    
    # The tech stocks we'll use for this analysis
    tech_list = [stock_name]

    dict_of_stocks = {}

    end = datetime.now()
    start = datetime(end.year - 10, end.month, end.day)

    for stock in tech_list:
        globals()[stock] = yf.download(stock, start, end)
        dict_of_stocks[tech_list[0]] = globals()[stock]
        

    return dict_of_stocks[tech_list[0]]

In [ ]:
main_df = get_stock_data("Kri.AT")

# 4 Inspecting the data

In [ ]:
main_df.tail()

# 4-1 Performance dataframe for models' error/evaluation

In [ ]:
model_performances = pd.DataFrame({
    'rmse': [np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN, np.NAN]
}, index=['Arima', 'rf', 'rf_tuned', 'XGBoost', 'XGBoost_tuned', 'svm', 'svm_tuned', 'kNN', 'kNN_tuned', 'LSTM'])

# 4-2 Dataset timestamps

In [ ]:
train_start_time = '2023-01-01'
train_end_time = '2023-08-31' 

validation_start_time = '2023-09-01'
validation_end_time = '2023-09-30'

test_start_time = '2023-10-01'

# 5 Statistical Learning - Time Series

In [ ]:
def split_dataset_for_arima(main_df, 
                            train_start = '2020-01-01',
                            train_end = '2022-12-31', 
                            validation_start = '2023-01-01',
                            validation_end = '2023-08-31',
                            test_start = '2023-09-01'):


    # Split the data into train, validation, and test sets
    train_set = main_df[(main_df.index >= train_start) & (main_df.index <= train_end)]
    validation_set = main_df[(main_df.index >= validation_start) & (main_df.index <= validation_end)]
    train_and_validation_set = pd.concat([train_set, validation_set], axis=0)
    test_set = main_df[main_df.index >= test_start]


    # Print the sizes of the sets
    print("Train Set Size:", len(train_set))
    print("Validation Set Size:", len(validation_set))
    print("Test Set Size:", len(test_set))

    return train_set, validation_set, train_and_validation_set, test_set

# 5-1. Spliting the dataset for Arima model

In [ ]:
train_set, validation_set, train_and_validation_set, test_set = split_dataset_for_arima(main_df, 
                                    train_start = train_start_time,
                                    train_end = train_end_time, 
                                    validation_start = validation_start_time,
                                    validation_end = validation_end_time,
                                    test_start = test_start_time)

# 6 Baseline Arima

## 6.1 Sources/tutorials

https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

https://www.section.io/engineering-education/multivariate-time-series-using-auto-arima/


## 6.2 Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(train_set.Close.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

If the p-value is less than a significance level (e.g., 0.05), you can reject the null hypothesis, indicating that the time series is stationary.

If the p-value is greater than the significance level, you fail to reject the null hypothesis, suggesting that the time series is non-stationary.

The p-value is 0.608091, which is greater than the common significance level of 0.05. Therefore, based on this ADF test, you fail to reject the null hypothesis, suggesting that your time series is likely non-stationary. Non-stationary time series often exhibit trends, seasonality, or other patterns that change over time.

## 6-3 How to find the order of differencing (d) in ARIMA model

In [ ]:
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(19,17), 'figure.dpi':120})


# Original Series
fig, axes = plt.subplots(3, 2)
axes[0, 0].plot(train_set.Close); axes[0, 0].set_title('Original Series')
plot_acf(train_set.Close, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(train_set.Close.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(train_set.Close.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(train_set.Close.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(train_set.Close.diff().diff().dropna(), ax=axes[2, 1])

plt.show()

## 6-4 Adf Test - KPSS test - PP test

In [ ]:
from pmdarima.arima.utils import ndiffs

## Adf Test
adf = ndiffs(train_set.Close, test='adf')  # 1

# KPSS test
kpss = ndiffs(train_set.Close, test='kpss')  # 1

# PP test:
pp = ndiffs(train_set.Close, test='pp')  # 1

adf, kpss, pp

## 6-5 PACF plot

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(19,8), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2)
axes[0].plot(train_set.Close.diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(train_set.Close.diff().dropna(), ax=axes[1])

plt.show()

## 6-6 Auto Arima to find the best parameters

find the optimal p, q, d orders

source / tutorial at:
https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

In [ ]:
import pmdarima as pm

auto_arima_model = pm.auto_arima(train_set['Close'], start_p=1, d=None, start_q=1, max_p=5, max_d=2, max_q=5, start_P=1, D=None, start_Q=1, max_P=2, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False,error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)

## 6-7 Build Arima model after finding AutoArima parameters

After using AutoArima, lets build the Arima model using statsmodels library

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model into the test set
df = pd.concat([validation_set[-1:], test_set])

arima_model = ARIMA(df['Close'], order=(auto_arima_model.order[0], 
                                        auto_arima_model.order[1], auto_arima_model.order[2]))

arima_model_res = arima_model.fit()
print(arima_model_res.summary())


# create forecasts
forecasts =  arima_model_res.predict(start=1, end=len(test_set))

# evaluate forecasts against test_set
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test_set['Close'], forecasts)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")


#plot Close and predicted Close price
plot_df = pd.DataFrame({'Date': test_set.index,
                    'Close': test_set['Close'],
                    'Predicted Close': forecasts
                        })


my_plot = sns.set(rc={'figure.figsize':(11, 3)})
plt.figure()
my_plot = sns.scatterplot(plot_df)
my_plot = sns.lineplot(plot_df)
my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
my_plot.set(title='Closing Price vs Predictions')

model_performances.loc['Arima'] = rmse



# 7 Multivariate Time Series


https://blogs.sap.com/2021/05/06/a-multivariate-time-series-modeling-and-forecasting-guide-with-python-machine-learning-client-for-sap-hana/

https://towardsdatascience.com/a-quick-introduction-on-granger-causality-testing-for-time-series-analysis-7113dc9420d2#:~:text=The%20Granger%20causality%20test%20is,predict%20tomorrow's%20Tesla's%20stock%20price%3F

## Grangercausalitytests

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=20
variables=train_set.columns  
matrix = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
for col in matrix.columns:
    for row in matrix.index:
        test_result = grangercausalitytests(main_df[[row, col]], maxlag=20, verbose=False)            
        p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]            
        min_p_value = np.min(p_values)
        matrix.loc[row, col] = min_p_value
matrix.columns = [var + '_x' for var in variables]
matrix.index = [var + '_y' for var in variables]
print(matrix)

From the result above, each column represents a predictor x of each variable and each row represents the response y and the p-value of each pair of variables are shown in the matrix. Take the value 0.0212 in (row 1, column 4) as an example, it refers that gdfco_x is causal to rgnp_y. Whereas, the 0.0 in (row 4, column 1) also refers to gdfco_y is the cause of rgnp_x. As all values are all below 0.05 except the diagonal, we could reject that the null hypothesis and this dataset is a good candidate of VectorARIMA modeling.

## 7-1 Ad fuller test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(df, series, sig=0.05):
    res = adfuller(df[series], autolag='AIC')    
    p_value = round(res[1], 3)

    if p_value <= sig:
        print(f" {series} : P-Value = {p_value} => Stationary. ")
    else:
        print(f" {series} : P-Value = {p_value} => Non-stationary.")

for column in train_set.columns:
    adfuller_test(main_df, column)

In [ ]:
data_differenced = train_set.diff().dropna()
for column in data_differenced.columns:
    adfuller_test(data_differenced, column)

In [ ]:
from statsmodels.tsa.api import VAR


model = VAR(data_differenced)
x = model.select_order(maxlags=21)
x.summary()

In [ ]:
from statsmodels.tsa.api import VAR

model = VAR(data_differenced)
aic = 999
best_lag = 0
for i in range(1,21):
    result = model.fit(i)
    if result.aic < aic:
        aic = result.aic
        best_lag = i

print("best lag: ", best_lag)

AIC drops at lag order 7. So it is better to keep lag upto 7

In [ ]:
model_fitted = model.fit(best_lag)
model_fitted.summary()

In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

 
for col, val in zip(data_differenced.columns, out):
    print(col, ':', round(val, 2))

In [ ]:
lag_order = model_fitted.k_ar
#print(lag_order)

# Input data for forecasting
data_differenced = train_set.diff()
forecast_input = data_differenced.values[-lag_order:]
#forecast_input

# Forecast
nobs = validation_set.shape[0]
fc = model_fitted.forecast(y=forecast_input, steps=nobs) # nobs defined at top of program
df_forecast = pd.DataFrame(fc, index=data_differenced.index[-nobs:], columns=data_differenced.columns + '_1d')
df_forecast

In [ ]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

df_results = invert_transformation(train_set, df_forecast, second_diff=False)
df_results

In [ ]:
fig, axes = plt.subplots(nrows=int(len(train_set.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(train_and_validation_set.columns, axes.flatten())):
    df_results[col+'_forecast'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    train_set[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();


# 8 Machine Learning models

# Prepare the closing price target variable

In [ ]:
main_df['target'] = main_df['Close'].shift(-1)
main_df.tail()

# 8-1 Split the dataset for ML

In [ ]:
def split_dataset_for_ml(main_df, 
                         train_start = '2023-01-01',
                         train_end = '2023-08-31', 
                         validation_start = '2023-09-01',
                         validation_end = '2023-09-30',
                         test_start = '2023-10-01'):
    
    
    # Prepare the closing price target variable    
    main_df['target'] = main_df['Close'].shift(-1)
    main_df.tail()


    # Split the data into train, validation, and test sets
    train_set = main_df[(main_df.index >= train_start) & (main_df.index <= train_end)]
    validation_set = main_df[(main_df.index >= validation_start) & (main_df.index <= validation_end)]
    train_and_validation_set = pd.concat([train_set, validation_set], axis=0)

    test_set = main_df[main_df.index >= test_start]
    test_set = test_set[:-1]


    # Print the sizes of the sets
    print("Train Set Size:", len(train_set))
    print("Validation Set Size:", len(validation_set))
    print("Test Set Size:", len(test_set))

    return train_set, validation_set, train_and_validation_set, test_set

In [ ]:
train_set, validation_set, train_and_validation_set, test_set = split_dataset_for_ml(main_df.copy(), 
                            train_start = train_start_time,
                            train_end = train_end_time, 
                            validation_start = validation_start_time,
                            validation_end = validation_end_time,
                            test_start = test_start_time)

# 8-3 Baseline Random Forest Model

In [ ]:
def build_random_forest_baseline_model(train_set, test_set):

    from sklearn.ensemble import RandomForestRegressor    
    
    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
    
    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)

    
    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']
    
    
    # Train the Random Forest model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=4)
    rf_model.fit(X_train, y_train)

    # Make predictions using the trained model
    y_pred = rf_model.predict(X_test)
    
    
    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    
    test_data["y_pred"] = y_pred
    
    
    # Calculate RMSE (Root Mean Squared Error) to evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(test_data["target"], test_data["y_pred"]))
    print(f"RMSE: {rmse}")


    #plot Close and predicted Close price
    plot_df = pd.DataFrame({'Date': test_data.index,
                       'Close': test_data['target'],
                       'Predicted Close': test_data['y_pred']
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')
    
    
    # Save the model performance and model's name
    model_performances.loc["rf"] = rmse
    
    return rf_model, rmse

rf_model_baseline, rf_model_rmse = build_random_forest_baseline_model(train_set.copy(), test_set.copy())

# 8-3-1 Tuned Random Forest

In [ ]:
def randomforest_tuning(train_set, validation_set, test_set):

    from hyperopt import hp, tpe, fmin, Trials, space_eval
    from sklearn.ensemble import RandomForestRegressor

    # Set a random seed
    ''' 
    By setting the random seed with np.random.seed(42), 
    you ensure that the random sampling in Hyperopt is reproducible across runs 
    as long as the seed remains the same.
    '''
    np.random.seed(42)

    # Define the search space for hyperparameters
    space = {
        'n_estimators': hp.choice('n_estimators', range(100, 200, 2)),
        'max_depth': hp.choice('max_depth', range(1, 6)),
        'min_samples_split': hp.choice('min_samples_split', range(2, 5)),
        'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 5)),
        'max_features': hp.choice('max_features', ['sqrt', 'log2'])
    }

    # Define the objective function to minimize (negative mean squared error)
    def objective(params):

        #scaling the data before modeling
        scaler = StandardScaler()
        scaled_array = scaler.fit_transform(train_set)
        train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
        
        scaled_array = scaler.transform(validation_set)
        validation_data = pd.DataFrame(scaled_array, columns=validation_set.columns, index=validation_set.index)

        
        # Prepare the data for Random Forest
        X_train = train_data.drop(['target'], axis=1, inplace=False)
        y_train = train_data['target']

        X_test = validation_data.drop(['target'], axis=1, inplace=False)
        y_test = validation_data['target']

        model = RandomForestRegressor(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            max_features=params['max_features'],
            random_state=42,
            n_jobs=4
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # unscale y_pred_diff
        train_data = train_set
        validation_data = validation_set
        y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
        y_test = validation_data['target']

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse

    # Create an instance of the Trials class to keep track of the results
    trials = Trials()

    # Run the hyperparameter search using the TPE algorithm
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=250,
                trials=trials,
                rstate=np.random.default_rng(42))

    # Get the best hyperparameters
    best_params = space_eval(space, best)
    print(best_params)

    # Train the final model with the best hyperparameters

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)

    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)


    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']


    final_model = RandomForestRegressor(**best_params, random_state=42)
    final_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    y_test = test_data['target']

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("{} rmse".format(rmse))


    #plot Close and predicted Close price
    plot_df = pd.DataFrame({'Date': test_data.index,
                       'Close': test_data['target'],
                       'Predicted Close': y_pred
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')

    # Save the model performance and model name
    model_performances.loc["rf_tuned"] = rmse

    return final_model, rmse

rf_model_tuned, rf_tuned_rmse = randomforest_tuning(train_set, validation_set, test_set)

# 8-4 Baseline XGBoost

In [ ]:
def build_xgboost_baseline_model(train_set, test_set):

    import xgboost as xgb
    
    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
    
    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)

    
    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']
    
    
    # Train the Random Forest model
    xgb_model = xgb.XGBRegressor(n_estimators=100)
    xgb_model.fit(X_train, y_train)

    # Make predictions using the trained model
    y_pred = xgb_model.predict(X_test)
    
    test_set["y_pred"] = y_pred
    
    
    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    
    test_data["y_pred"] = y_pred
    
    
    # Calculate RMSE (Root Mean Squared Error) to evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(test_data["target"], test_data["y_pred"]))
    print(f"RMSE: {rmse}")


    #plot Close and predicted Close price
    
    plot_df = pd.DataFrame({'Date': test_set.index,
                       'Close': test_set['target'],
                       'Predicted Close': test_set['y_pred']
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')
    
    # Save the model performance and model name
    model_performances.loc["XGBoost"] = rmse
    
    return xgb_model, rmse

xgb_model_baseline, xgb_model_rmse = build_xgboost_baseline_model(train_set.copy(), test_set.copy())

# 8-4-1 Tuned XGBoost

In [ ]:
def xgboost_tuning(train_set, validation_set, test_set):

    from hyperopt import hp, tpe, fmin, Trials, space_eval
    import xgboost as xgb

    # Set a random seed
    ''' 
    By setting the random seed with np.random.seed(42), 
    you ensure that the random sampling in Hyperopt is reproducible across runs 
    as long as the seed remains the same.
    '''
    np.random.seed(42)

    # Define the search space for hyperparameters
    space = {
        'n_estimators': hp.choice('n_estimators', range(100, 400, 2)),
        'max_depth': hp.choice('max_depth', range(1, 15)),
        'learning_rate':  hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'min_child_weight': hp.choice('min_child_weight', range(1, 11)),
        'gamma': hp.quniform('gamma', 0.4, 1, 0.05),
    }

    # Define the objective function to minimize (negative mean squared error)
    def objective(params):

        #scaling the data before modeling
        scaler = StandardScaler()
        scaled_array = scaler.fit_transform(train_set)
        train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
        
        scaled_array = scaler.transform(validation_set)
        validation_data = pd.DataFrame(scaled_array, columns=validation_set.columns, index=validation_set.index)

        
        # Prepare the data for Random Forest
        X_train = train_data.drop(['target'], axis=1, inplace=False)
        y_train = train_data['target']

        X_test = validation_data.drop(['target'], axis=1, inplace=False)
        y_test = validation_data['target']

        model = xgb.XGBRegressor(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            subsample=params['subsample'],
            min_child_weight=params['min_child_weight'],
            gamma=params['gamma'],
            objective='reg:squarederror'
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # unscale y_pred_diff
        train_data = train_set
        validation_data = validation_set
        y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
        y_test = validation_data['target']

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse

    # Create an instance of the Trials class to keep track of the results
    trials = Trials()

    # Run the hyperparameter search using the TPE algorithm
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=250,
                trials=trials,
                rstate=np.random.default_rng(42))

    # Get the best hyperparameters
    best_params = space_eval(space, best)
    print(best_params)

    # Train the final model with the best hyperparameters

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)

    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)


    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']


    final_model = xgb.XGBRegressor(**best_params, random_state=42)
    final_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    y_test = test_data['target']

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("{} rmse".format(rmse))


    #plot Close and predicted Close price
    plot_df = pd.DataFrame({'Date': test_data.index,
                       'Close': test_data['target'],
                       'Predicted Close': y_pred
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')


    # Save the model performance and model name
    model_performances.loc["XGBoost_tuned"] = rmse


    return final_model, rmse

xgboost_model_tuned, xgboost_tuned_rmse = xgboost_tuning(train_set, validation_set, test_set)

# 8-5 Baseline SVM

In [ ]:
def build_svm_baseline_model(train_set, test_set):
    
    from sklearn.svm import SVR

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
    
    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)

    
    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']
    
    
    # Train the Random Forest model
    svm = SVR(kernel='linear')
    svm.fit(X_train, y_train)

    # Make predictions using the trained model
    y_pred = svm.predict(X_test)
    
    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    
    test_data["y_pred"] = y_pred
    
    
    # Calculate RMSE (Root Mean Squared Error) to evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(test_data["target"], test_data["y_pred"]))
    print(f"RMSE: {rmse}")


    #plot Close and predicted Close price
    
    plot_df = pd.DataFrame({'Date': test_set.index,
                       'Close': test_set['target'],
                       'Predicted Close': test_set['y_pred']
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')
    
    # Save the model performance and model's name
    model_performances.loc["svm"] = rmse
    
    return svm, rmse

svm_model_baseline, svm_model_rmse = build_svm_baseline_model(train_set.copy(), test_set.copy())

# 8-5-1 Tuned SVM

In [ ]:
def svr_tuning(train_set, validation_set, test_set):

    from hyperopt import hp, tpe, fmin, Trials, space_eval
    from sklearn.svm import SVR

    # Set a random seed
    ''' 
    By setting the random seed with np.random.seed(42), 
    you ensure that the random sampling in Hyperopt is reproducible across runs 
    as long as the seed remains the same.
    '''
    np.random.seed(42)

    # Define the search space for hyperparameters
    space = {
        'kernel': hp.choice('kernel', ['linear', 'rbf', 'sigmoid']),
        'degree':  hp.choice('degree', range(1, 6)),
        'C': hp.uniform('C', 0, 1),  # C parameter (log-uniform search space)
        'epsilon': hp.loguniform('epsilon', -4, 1),  # Epsilon (log-uniform search space)

    }

    # Define the objective function to minimize (negative mean squared error)
    def objective(params):

        #scaling the data before modeling
        scaler = StandardScaler()
        scaled_array = scaler.fit_transform(train_set)
        train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
        
        scaled_array = scaler.transform(validation_set)
        validation_data = pd.DataFrame(scaled_array, columns=validation_set.columns, index=validation_set.index)

        
        # Prepare the data for Random Forest
        X_train = train_data.drop(['target'], axis=1, inplace=False)
        y_train = train_data['target']

        X_test = validation_data.drop(['target'], axis=1, inplace=False)
        y_test = validation_data['target']

        model = SVR(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # unscale y_pred_diff
        train_data = train_set
        validation_data = validation_set
        y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
        y_test = validation_data['target']

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse

    # Create an instance of the Trials class to keep track of the results
    trials = Trials()

    # Run the hyperparameter search using the TPE algorithm
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=250,
                trials=trials,
                rstate=np.random.default_rng(42))


    # Get the best hyperparameters
    best_params = space_eval(space, best)
    print(best_params)

    # Train the final model with the best hyperparameters

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)

    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)


    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']


    final_model = SVR(**best_params)
    final_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    y_test = test_data['target']

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("{} rmse".format(rmse))

    #plot Close and predicted Close price
    plot_df = pd.DataFrame({'Date': test_data.index,
                       'Close': test_data['target'],
                       'Predicted Close': y_pred
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')


    # Save the model performance and model name
    model_performances.loc["svm_tuned"] = rmse


    return final_model, rmse

svm_model_tuned, svm_tuned_rmse = svr_tuning(train_set, validation_set, test_set)

# 8-6 Baseline kNN

In [ ]:
def build_knn_baseline_model(train_set, test_set):
    
    from sklearn.neighbors import KNeighborsRegressor

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
    
    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)

    
    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']
    
    
    # Train the Random Forest model
    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(X_train, y_train)

    # Make predictions using the trained model
    y_pred = knn.predict(X_test)
    
    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    
    test_data["y_pred"] = y_pred
    
    
    # Calculate RMSE (Root Mean Squared Error) to evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(test_data["target"], test_data["y_pred"]))
    print(f"RMSE: {rmse}")


    #plot Close and predicted Close price
    
    plot_df = pd.DataFrame({'Date': test_set.index,
                       'Close': test_set['target'],
                       'Predicted Close': test_set['y_pred']
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')
    
    # Save the model performance and model's name
    model_performances.loc["kNN"] = rmse

    # Save the model performance and model name
    model_performances.loc["kNN_tuned"] = rmse
    
    return knn, rmse

knn_model_baseline, knn_model_rmse = build_knn_baseline_model(train_set.copy(), test_set.copy())

# 8-6-1 Tuned kNN

In [ ]:
def kNN_tuning(train_set, validation_set, test_set):

    from hyperopt import hp, tpe, fmin, Trials, space_eval
    from sklearn.neighbors import KNeighborsRegressor

    # Set a random seed
    ''' 
    By setting the random seed with np.random.seed(42), 
    you ensure that the random sampling in Hyperopt is reproducible across runs 
    as long as the seed remains the same.
    '''
    np.random.seed(42)

    # Define the search space for hyperparameters
    space = {
        'n_neighbors': hp.choice('n_neighbors', range(1, 21)),  # Number of neighbors (integer values)
        'weights': hp.choice('weights', ['uniform', 'distance']),
        'p': hp.quniform('p', 1, 2, 1),  # Minkowski power parameter (integer values)
    }

    # Define the objective function to minimize (negative mean squared error)
    def objective(params):

        #scaling the data before modeling
        scaler = StandardScaler()
        scaled_array = scaler.fit_transform(train_set)
        train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
        
        scaled_array = scaler.transform(validation_set)
        validation_data = pd.DataFrame(scaled_array, columns=validation_set.columns, index=validation_set.index)

        
        # Prepare the data for Random Forest
        X_train = train_data.drop(['target'], axis=1, inplace=False)
        y_train = train_data['target']

        X_test = validation_data.drop(['target'], axis=1, inplace=False)
        y_test = validation_data['target']

        model = KNeighborsRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # unscale y_pred_diff
        train_data = train_set
        validation_data = validation_set
        y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
        y_test = validation_data['target']

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse

    # Create an instance of the Trials class to keep track of the results
    trials = Trials()

    # Run the hyperparameter search using the TPE algorithm
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=250,
                trials=trials,
                rstate=np.random.default_rng(42))


    # Get the best hyperparameters
    best_params = space_eval(space, best)
    print(best_params)

    # Train the final model with the best hyperparameters

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)

    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)


    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']


    final_model = KNeighborsRegressor(**best_params)
    final_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test)

    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    y_test = test_data['target']

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("{} rmse".format(rmse))


    #plot Close and predicted Close price
    plot_df = pd.DataFrame({'Date': test_data.index,
                       'Close': test_data['target'],
                       'Predicted Close': y_pred
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')

    return final_model, rmse

knn_model_tuned, knn_tuned_rmse = kNN_tuning(train_set, validation_set, test_set)

# 8-7 Baseline LSTM

In [ ]:
def build_lstm_baseline_model(train_set, test_set):
    
    from tensorflow import keras

    # Set a random seed
    np.random.seed(42)

    #scaling the data before modeling
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(train_set)
    train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)
    
    scaled_array = scaler.transform(test_set)
    test_data = pd.DataFrame(scaled_array, columns=test_set.columns, index=test_set.index)

    
    # Prepare the data for Random Forest
    X_train = train_data.drop(['target'], axis=1, inplace=False)
    y_train = train_data['target']

    X_test = test_data.drop(['target'], axis=1, inplace=False)
    y_test = test_data['target']
    
    
    # build the LSTM model
    lstm = keras.Sequential()
    lstm.add(keras.layers.LSTM(20, input_shape=(X_train.shape[1], 1)))
    lstm.add(keras.layers.Dense(1))
    
    # Compile the model
    lstm.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train the Model
    lstm.fit(X_train, y_train, epochs=20, batch_size=30, verbose=0)

    # Make predictions using the trained model
    y_pred = lstm.predict(X_test)
    
    # unscale y_pred_diff
    train_data = train_set
    test_data = test_set
    y_pred = (y_pred * train_data['Close'].std()) + train_data['Close'].mean()
    
    test_data["y_pred"] = y_pred
    
    
    # Calculate RMSE (Root Mean Squared Error) to evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(test_data["target"], test_data["y_pred"]))
    print(f"RMSE: {rmse}")


    #plot Close and predicted Close price
    
    plot_df = pd.DataFrame({'Date': test_set.index,
                       'Close': test_set['target'],
                       'Predicted Close': test_set['y_pred']
                           })


    my_plot = sns.set(rc={'figure.figsize':(11, 3)})
    plt.figure()
    my_plot = sns.scatterplot(plot_df)
    my_plot = sns.lineplot(plot_df)
    my_plot.set_xticklabels(my_plot.get_xticklabels(), rotation=45)
    my_plot.set(title='Closing Price vs Predictions')
    
    # Save the model performance and model name
    model_performances.loc["LSTM"] = rmse
    
    return lstm, rmse

lstm_model_baseline, lstm_model_rmse = build_lstm_baseline_model(train_set.copy(), test_set.copy())

# 9 Evaluate Models' performances

In [ ]:
df_model_performances = pd.DataFrame(model_performances)
print(df_model_performances)

# Save the dataframe to a CSV file
df_model_performances.to_csv('model_performances.csv', index=False)

# Find the best lag value

This is a simple experiment.

In [ ]:
def get_series_with_lag(train_set, validation_set, model_str='SVM'):
    
    import pandas as pd
    import numpy as np
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.svm import SVR
    import xgboost as xgb
    from sklearn.neighbors import KNeighborsRegressor
    
    from sklearn.metrics import mean_squared_error
    from sklearn.preprocessing import MinMaxScaler

    # Specify the maximum lag period to consider
    max_lag_period = 20

    # Create empty lists to store lag periods and corresponding RMSE scores
    lag_periods = []
    rmse_scores = []
    
    features = train_set.columns
    print(model_str)

    # Create lag features and evaluate RMSE for each lag period
    for lag_period in range(0, max_lag_period + 1):
        if lag_period == 0:
            
            X_train = train_set.drop(['target'], axis=1)
            y_train = train_set['target']
            X_valid = validation_set.drop(['target'], axis=1)
            y_valid = validation_set['target']  
            
        if lag_period > 0:
        # Create lag features
            for feature in features:
                train_set['{}_Lag_{}'.format(feature, lag_period)] = train_set[feature].shift(lag_period)
                validation_set['{}_Lag_{}'.format(feature, lag_period)] = validation_set[feature].shift(lag_period)
                
                # fill NaN values
                train_set.fillna(0, inplace=True)
                validation_set.fillna(0, inplace=True)
                
        #scaling the data before modeling
        scaler = StandardScaler()
        scaled_array = scaler.fit_transform(train_set)
        train_data = pd.DataFrame(scaled_array, columns=train_set.columns, index=train_set.index)

        scaled_array = scaler.transform(validation_set)
        validation_data = pd.DataFrame(scaled_array, columns=validation_set.columns, index=validation_set.index)

        X_train = train_data.drop(['target'], axis=1)
        y_train = train_data['target']
        X_valid = validation_data.drop(['target'], axis=1)
        y_valid = validation_data['target']
                
        
        # Train a model
        if model_str=='SVM':
            model = SVR(kernel='linear')
            model.fit(X_train, y_train)
        elif model_str=="RandomForest":
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
        elif model_str=="XGBoost":
            model = xgb.XGBRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
        elif model_str=="kNN":
            model = KNeighborsRegressor(n_neighbors=3)
            model.fit(X_train, y_train)
        else:
            print("Please select a model like: 'SVM', 'RandomForest', 'XGBoost', 'kNN'")
            return None

        # Make predictions on the validation set
        y_pred = model.predict(X_valid)
        
        
        # unscale y_pred
        train_data = train_set
        validation_data = validation_set
        y_pred = (y_pred * train_data['target'].std()) + train_data['target'].mean()

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(validation_data['target'], y_pred))
        print("{} rmse with lag {}".format(rmse, lag_period))

        # Store lag period and RMSE
        lag_periods.append(lag_period)
        rmse_scores.append(rmse)

    # Find the best lag period with the lowest RMSE
    best_lag_period = lag_periods[np.argmin(rmse_scores)]
    best_rmse = rmse_scores[np.argmin(rmse_scores)]

    print("Best Lag Period: {}".format(best_lag_period))
    print("Best RMSE: {}".format(best_rmse))

    return best_lag_period, best_rmse

In [ ]:
# best_lag_period, best_rmse = get_series_with_lag(train_set.copy(), validation_set.copy(), "RandomForest")

In [ ]:
# best_lag_period, best_rmse = get_series_with_lag(train_set.copy(), validation_set.copy(), "SVM")

In [ ]:
# best_lag_period, best_rmse = get_series_with_lag(train_set.copy(), validation_set.copy(), "XGBoost")

In [ ]:
# best_lag_period, best_rmse = get_series_with_lag(train_set.copy(), validation_set.copy(), "kNN")

# Save the models

In [ ]:
import pickle

pickle.dump(knn_model_baseline, open('knn_model_baseline.pkl', 'wb'))
pickle.dump(svm_model_baseline, open('svm_model_baseline.pkl', 'wb'))
pickle.dump(xgb_model_baseline, open('xgb_model_baseline.pkl', 'wb'))
pickle.dump(rf_model_baseline, open('rf_model_baseline.pkl', 'wb'))